# Model2
This notebook is a deeper iteration of similar architecture from model 1 set up for the purpose of running on AWS.

In [ ]:
import sagemaker
import tensorflow as tf
from sagemaker.tensorflow import TensorFlow

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
bucket = 'art-descriptor-paintings'
pkl_dir = f's3://{bucket}/PKL'

## Import Data

In [ ]:
desc_path        = os.path.join(pkl_dir, 'condensed_descriptions.pkl')
feat_path        = os.path.join(pkl_dir, 'full_features.pkl')
train_list_path  = os.path.join(pkl_dir, 'train_list_full.pkl')
val_list_path    = os.path.join(pkl_dir, 'val_list_full.pkl')
test_list_path   = os.path.join(pkl_dir, 'test_list.pkl')
test_art_path    = os.path.join(pkl_dir, 'test_list_art.pkl')

with open(train_list_path, 'rb') as fp:
    train_list_full = pickle.load(fp)

with open(val_list_path, 'rb') as fp:
    val_list_full = pickle.load(fp)

with open(test_list_path, 'rb') as fp:
    test_list = pickle.load(fp)

with open(test_art_path, 'rb') as fp:
    test_list_art = pickle.load(fp)

with open(desc_path, 'rb') as fp:
    descriptions = pickle.load(fp)

with open(feat_path, 'rb') as fp:
    features = pickle.load(fp)

In [ ]:
from SCRIPT.sequence_generator import *

In [ ]:
processor = sequence_generator(descriptions, features)

train_X1, train_X2, train_Y = processor.train_generator(train_list_full)
val_X1, val_X2, val_Y = processor.validation_generator(val_list_full)

# get params
tokenizer = processor.get_tokenizer()
max_length = processor.get_max_length()
num_vocab = processor.get_num_vocab()

## Local Mode

In [ ]:
batch_size = 128
epochs = 1

In [ ]:
tf_estimator = TensorFlow(entry_point = 'SCRIPT/nlg_model_tf_1.py', 
                         role = role, 
                         train_instance_count = 1, 
                         train_instance_type = 'local', 
                         py_version = 'py37', 
                         framework_version = '1.15.2',
                         script_mode = True, 
                         hyperparameters = {'epochs': epochs, 
                                           'batch-size': batch_size}
                         )

In [ ]:
tf_estimator.fit({'pkl': pkl_dir})

In [ ]:
tf_estimator = TensorFlow(entry_point = 'SCRIPT/nlg_model_tf_1.py', 
                         role = role, 
                         train_instance_count = 1, 
                         train_instance_type = '',
                         py_version = 'py37', 
                         framework_version = '1.15.2',
                         script_mode = True, 
                         hyperparameters = {'epochs': epochs, 
                                           'batch-size': batch_size}
                         )

In [ ]:
tf_estimator.fit({'pkl': pkl_dir})

## Deploy

In [ ]:
import time

tf_endpoint = 'nlg-model-tf-'+time.strftime('%Y-%m-%d-%H-%M-%S', time.gmtime())

# GPU
#tf_predictor = tf_estimator.deploy(initial_instance_count = 1, 
#                                   instance_type = 'ml.p2.xlarge')


tf_predictor = tf_estimator.deploy(initial_instance_count = 1, 
                                  instance_type = 'ml.c5.large', 
                                  accelerator_type = 'ml.eia1.medium', 
                                  endpoint_name = tf_endpoint)

## Evaluation

In [ ]:
from SCRIPT.evaluation_tools import *

In [ ]:
# testing on photo data
pred_list, score = get_bleu(test_list, 
                            features, 
                            tokenizer, 
                            max_length, 
                            tf_predictor, 
                            descriptions)

In [ ]:
# testing on photo data
artpred_list, artscore = get_bleu(test_list_art, 
                            features, 
                            tokenizer, 
                            max_length, 
                            tf_predictor, 
                            descriptions)

In [ ]:
#flicker_descriptor = descriptor(features, tokenizer, processor, model)
#flicker_descriptor.test_random_image(img_id)

In [ ]:
#art_descriptor = descriptor(features, tokenizer, processor, model,img_dir = art_dir)
#art_descriptor.test_one_image(img_id)

## Delete End-point

In [ ]:
sess.delete_endpoint(endpoint_name = tf_endpoint)